In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from datetime import timedelta

import plotly.offline
import cufflinks as cf
import plotly.express as px
from plotly.offline import iplot
import plotly.graph_objects as go
from IPython.core.interactiveshell import InteractiveShell

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# def make_date(year: str, month: str, day: str, time: str):
#     return f'{year}-{month}-{day} {time[:-1]}'

# main_df = pd.DataFrame()
# for file in tqdm(files):
#     df = pd.read_csv(f'{path}/{file}')
#     year = file[:4]
#     month = file[4:6]
#     day = file[6:8]
#     df['datetime'] = df['datetime'].apply(lambda x: make_date(year, month, day, x))
#     #main_df = pd.concat([main_df, df])
#     df.to_csv(f'/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/BTCUSDT/BINANCE/Data/{file}.csv',index=False)

In [3]:
def get_columns_indexes(orderbook_levels: int, needed_orderbook_levels: int):
    columns = []
    for i in range(2, 4):
        for j in range(orderbook_levels):
            if j < needed_orderbook_levels:
                columns.append(j + (i * orderbook_levels))
    columns.append(orderbook_levels * 4)
    
    return columns

In [4]:
path = '/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/BTCUSDT/BINANCE/Data'
files = sorted(os.listdir(path))

In [5]:
for file in tqdm(sorted(os.listdir(f'{path}'))):
    df = pd.read_csv(f'{path}/{file}')
    orderbook_levels = 30
    freq = '1s' 
    columns = get_columns_indexes(orderbook_levels, orderbook_levels)
    mid_price = (df['ask_0_price']+df['bid_0_price'])/2
    df = df.iloc[:, columns]
    df['mid_price'] = mid_price
    df = df[df['mid_price'] > 0]
    df.index = df['datetime'].values
    df['datetime'] = pd.to_datetime(df['datetime'])
    df_Bt_bars = pd.DataFrame(columns=['datetime','Mean price',])
    df_Bt_bars['datetime'] = df.groupby(pd.Grouper(key='datetime', freq=freq)).last().index
    df_Bt_bars['Mean price'] = df.groupby(pd.Grouper(key='datetime', freq=freq)).last()['mid_price'].values
    for i in range(2):
        for j in range(orderbook_levels):
            if i == 0:
                df_Bt_bars[f'bid_{j}_size'] = df.groupby(pd.Grouper(key='datetime', freq=freq)).sum()[f'bid_{j}_size'].values
            elif i == 1:
                df_Bt_bars[f'ask_{j}_size'] = df.groupby(pd.Grouper(key='datetime', freq=freq)).sum()[f'ask_{j}_size'].values
                
    filename = file[:-4]
    df_Bt_bars.to_csv(f'/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/BTCUSDT/BINANCE/Bars/{file}',index=False)

100%|██████████| 31/31 [10:04<00:00, 19.50s/it]


In [16]:
path = '/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/BTCUSDT/BINANCE/Bars'
files = sorted(os.listdir(path))
main_df = pd.DataFrame()
for file in files:     
    file_df = pd.read_csv(f'{path}/{file}')
    main_df = pd.concat([main_df, file_df], ignore_index=True)
    print(f'{file}')
main_df['datetime'] = pd.to_datetime(main_df['datetime'])
main_df.index = range(len(main_df))

20201201.csv
20201202.csv
20201203.csv
20201204.csv
20201205.csv
20201206.csv
20201207.csv
20201208.csv
20201209.csv
20201210.csv
20201211.csv
20201212.csv
20201213.csv
20201214.csv
20201215.csv
20201216.csv
20201217.csv
20201218.csv
20201219.csv
20201220.csv
20201221.csv
20201222.csv
20201223.csv
20201224.csv
20201225.csv
20201226.csv
20201227.csv
20201228.csv
20201229.csv
20201230.csv
20201231.csv


In [19]:
main_df.to_csv('/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/Bars_20.12.csv')